<a href="https://colab.research.google.com/github/oktaviacitra/data-mining/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn import tree, metrics
from sklearn.tree import DecisionTreeClassifier

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read Dataset

In [ ]:
path = "/content/drive/MyDrive/Data Mining/"

In [ ]:
train_data = pd.read_csv(path + "titanic.csv")

In [ ]:
train_label = train_data.loc[:, ['Survived']]
train_data = train_data.drop(['Survived'], axis=1)

In [ ]:
test_data = pd.read_csv(path + "titanic_test.csv")

In [ ]:
test_label = pd.read_csv(path + "titanic_testlabel.csv").drop(['PassengerId'], axis=1)

# Delete Some Features

In [ ]:
train_data = train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_data = test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Change Categorical Features

In [ ]:
train_data = train_data.replace({'female':1, 'male':0, 'S':0, 'C':1, 'Q':2})
test_data = test_data.replace({'female':1, 'male':0, 'S':0, 'C':1, 'Q':2})

# Drop Missing Values

In [ ]:
pos_missing_train = [index for index, row in train_data.iterrows() if row.isnull().any()]
train_data = train_data.dropna()
train_label = train_label.drop(pos_missing_train)
train_data['Embarked'] = train_data['Embarked'].astype(int)

In [ ]:
pos_missing_test = [index for index, row in test_data.iterrows() if row.isnull().any()]
test_data = test_data.dropna()
test_label = test_label.drop(pos_missing_test)

In [ ]:
train_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...
885,3,1,39.0,0,5,29.1250,2
886,2,0,27.0,0,0,13.0000,0
887,1,1,19.0,0,0,30.0000,0
889,1,0,26.0,0,0,30.0000,1


In [ ]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0
...,...,...,...,...,...,...,...
409,3,1,3.0,1,1,13.7750,0
411,1,1,37.0,1,0,90.0000,2
412,3,1,28.0,0,0,7.7750,0
414,1,1,39.0,0,0,108.9000,1


# Data Normalization

In [ ]:
new_min, new_max = 0, 1
new_min_max = []
for column in train_data.columns:
  if column == 'Age' or column == 'Fare':
    data = train_data[column]
    min = data.min()
    max = data.max()
    new_min_max.append([min, max])
    train_data[column] = (data - min) * (new_max - new_min) / (max - min) + new_min
train_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,0.271174,1,0,0.014151,0
1,1,1,0.472229,1,0,0.139136,1
2,3,1,0.321438,0,0,0.015469,0
3,1,1,0.434531,1,0,0.103644,0
4,3,0,0.434531,0,0,0.015713,0
...,...,...,...,...,...,...,...
885,3,1,0.484795,0,5,0.056848,2
886,2,0,0.334004,0,0,0.025374,0
887,1,1,0.233476,0,0,0.058556,0
889,1,0,0.321438,0,0,0.058556,1


In [ ]:
i = 0
for column in test_data.columns:
  if column == 'Age' or column == 'Fare':
    data = test_data[column]
    min = new_min_max[i][0]
    max = new_min_max[i][1]
    test_data[column] = (data - min) * (new_max - new_min) / (max - min) + new_min
    i += 1
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,0.428248,0,0,0.015282,2
1,3,1,0.585323,1,0,0.013663,0
2,2,0,0.773813,0,0,0.018909,2
3,3,0,0.334004,0,0,0.016908,0
4,3,1,0.271174,1,1,0.023984,0
...,...,...,...,...,...,...,...
409,3,1,0.032420,1,1,0.026887,0
411,1,1,0.459663,1,0,0.175668,2
412,3,1,0.346569,0,0,0.015176,0
414,1,1,0.484795,0,0,0.212559,1


# Classification

In [ ]:
class Record:
  def __init__(self, columns, error_ratio):
    self.columns = columns
    self.error_ratio = error_ratio

## Select Features

### 2

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
records = []
for i in range(0, 6):
  for j in range(i, 6):
    columns = []
    if i != j:
      columns.append(train_data.columns[i])
      columns.append(train_data.columns[j])
      train_data_2 = train_data.loc[:, columns]
      test_data_2 = test_data.loc[:, columns]
      clf_2 = dtc.fit(train_data_2, train_label)
      pred_label = clf_2.predict(test_data_2)
      acc = metrics.accuracy_score(test_label, pred_label)
      err = round((1 - acc) * 100, 2)
      records.append(Record(columns, err))

## 3

In [ ]:
for i in range(0, 6):
  for j in range(i, 6):
    for k in range(j, 6):
      columns = []
      if i != j and i != k and j != k:
        columns.append(train_data.columns[i])
        columns.append(train_data.columns[j])
        columns.append(train_data.columns[k])
        train_data_3 = train_data.loc[:, columns]
        test_data_3 = test_data.loc[:, columns]
        clf_3 = dtc.fit(train_data_3, train_label)
        pred_label = clf_3.predict(test_data_3)
        acc = metrics.accuracy_score(test_label, pred_label)
        err = round((1 - acc) * 100, 2)
        records.append(Record(columns, err))

## 4

In [ ]:
for i in range(0, 6):
  for j in range(i, 6):
    for k in range(j, 6):
      for l in range(k, 6):
        columns = []
        if i != j and i != k and i != l and j != l and k != l and j != k:
          columns.append(train_data.columns[i])
          columns.append(train_data.columns[j])
          columns.append(train_data.columns[k])
          columns.append(train_data.columns[l])
          train_data_4 = train_data.loc[:, columns]
          test_data_4 = test_data.loc[:, columns]
          clf_4 = dtc.fit(train_data_4, train_label)
          pred_label = clf_4.predict(test_data_4)
          acc = metrics.accuracy_score(test_label, pred_label)
          err = round((1 - acc) * 100, 2)
          records.append(Record(columns, err))

## 5

In [ ]:
for i in range(0, 6):
  for j in range(i, 6):
    for k in range(j, 6):
      for l in range(k, 6):
        for m in range(l, 6):
          columns = []
          if i != j and i != k and i != l and j != l and k != l and j != k and i != m and j != m and k != m and l != m:
            columns.append(train_data.columns[i])
            columns.append(train_data.columns[j])
            columns.append(train_data.columns[k])
            columns.append(train_data.columns[l])
            columns.append(train_data.columns[m])
            train_data_5 = train_data.loc[:, columns]
            test_data_5 = test_data.loc[:, columns]
            clf_5 = dtc.fit(train_data_5, train_label)
            pred_label = clf_5.predict(test_data_5)
            acc = metrics.accuracy_score(test_label, pred_label)
            err = round((1 - acc) * 100, 2)
            records.append(Record(columns, err))

## 6

In [ ]:
clf_6 = dtc.fit(train_data, train_label)
pred_label = clf_6.predict(test_data)
acc = metrics.accuracy_score(test_label, pred_label)
err = round((1 - acc) * 100, 2)
records.append(Record(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], err))

In [ ]:
for record in records:
  print(record.columns, " ", record.error_ratio)

['Pclass', 'Sex']   15.11
['Pclass', 'Age']   41.69
['Pclass', 'SibSp']   39.88
['Pclass', 'Parch']   37.16
['Pclass', 'Fare']   38.07
['Sex', 'Age']   10.27
['Sex', 'SibSp']   0.91
['Sex', 'Parch']   0.3
['Sex', 'Fare']   11.18
['Age', 'SibSp']   41.69
['Age', 'Parch']   40.48
['Age', 'Fare']   43.5
['SibSp', 'Parch']   34.44
['SibSp', 'Fare']   36.86
['Parch', 'Fare']   35.65
['Pclass', 'Sex', 'Age']   17.52
['Pclass', 'Sex', 'SibSp']   6.04
['Pclass', 'Sex', 'Parch']   13.29
['Pclass', 'Sex', 'Fare']   11.78
['Pclass', 'Age', 'SibSp']   43.5
['Pclass', 'Age', 'Parch']   44.71
['Pclass', 'Age', 'Fare']   43.5
['Pclass', 'SibSp', 'Parch']   38.07
['Pclass', 'SibSp', 'Fare']   36.25
['Pclass', 'Parch', 'Fare']   35.95
['Sex', 'Age', 'SibSp']   13.6
['Sex', 'Age', 'Parch']   17.52
['Sex', 'Age', 'Fare']   18.43
['Sex', 'SibSp', 'Parch']   5.14
['Sex', 'SibSp', 'Fare']   11.48
['Sex', 'Parch', 'Fare']   11.78
['Age', 'SibSp', 'Parch']   41.39
['Age', 'SibSp', 'Fare']   43.81
['Age', 'Par

In [ ]:
loc = 0
for i in range(0, len(records)):
  if records[i].error_ratio < records[loc].error_ratio:
    loc = i
print(records[loc].columns, " ", records[loc].error_ratio)

['Sex', 'Parch']   0.3
